In [59]:
import pandas as pd
import numpy as np

import mlxtend
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

import xlsxwriter #The XlsxWriter libarary for 

In [2]:
df = pd.read_csv('sales_2018-01-01_2021-08-24.csv')
df.head(5)

,month,product_type,order_id,order_name,net_sales,ordered_item_quantity
0,2018-01,NaN,147056590858,SL273228,0.00,0
1,2018-01,Wide Headband,224372391946,SL276292,2.50,1
2,2018-01,NaN,236564316170,SL278079,0.00,0
3,2018-01,Wide Headband,221263462410,SL275671,5.26,1
4,2018-01,NaN,226704097290,SL276606,0.00,0


In [3]:
df.shape

(1276459, 6)

In [4]:
# Removing irrelevant data
df = df[df['net_sales']!=0]
df = df[df['ordered_item_quantity']!=0]

df.head(5)

,month,product_type,order_id,order_name,net_sales,ordered_item_quantity
1,2018-01,Wide Headband,224372391946,SL276292,2.50,1
3,2018-01,Wide Headband,221263462410,SL275671,5.26,1
7,2018-01,Wide Headband,217893765130,SL275248,2.00,1
9,2018-01,Wide Headband,236094816266,SL277815,33.35,5
11,2018-01,Wide Headband,151183261706,SL274151,13.30,3


In [5]:
df[['product_type']].describe()

,product_type
count,738371
unique,132
top,Sleeve
freq,98879


In [6]:
# unique 'product_type'
df['product_type'].value_counts(sort=True)

Sleeve                  98879
Wide Headband           87422
Neck Gaiter             57707
Spats                   54295
Mask With Nose Shape    36581
                        ...  
Visor Decal                 9
Shorts sleeve Jersey        8
Over Knee Socks             1
Basics Shirt                1
Racerback                   1
Name: product_type, Length: 132, dtype: int64

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 738523 entries, 1 to 1276458
Data columns (total 6 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   month                  738523 non-null  object 
 1   product_type           738371 non-null  object 
 2   order_id               738523 non-null  int64  
 3   order_name             738523 non-null  object 
 4   net_sales              738523 non-null  float64
 5   ordered_item_quantity  738523 non-null  int64  
dtypes: float64(1), int64(2), object(3)
memory usage: 39.4+ MB


In [8]:
# check amount of NAs
df.isna().sum()

month                      0
product_type             152
order_id                   0
order_name                 0
net_sales                  0
ordered_item_quantity      0
dtype: int64

In [9]:
# certain items don't have 'prodyc_type'
df[df['product_type'].isna()]

,month,product_type,order_id,order_name,net_sales,ordered_item_quantity
9231,2018-01,NaN,147777060874,SL273418,60.0,20
26865,2018-02,NaN,244497743882,SL279803,8.0,1
35270,2018-03,NaN,289633337354,SL286475,5.0,1
54753,2018-04,NaN,459475157112,SL294639,40.0,2
62483,2018-05,NaN,481809203320,SL296291,45.0,3
...,...,...,...,...,...,...
1222534,2021-07,NaN,3879108018269,SL742413,50.0,1
1237418,2021-08,NaN,3916056035421,SL760246,12.0,1
1237742,2021-08,NaN,3922659672157,SL764102,20.0,1
1253049,2021-08,NaN,3889200758877,SL746318,12.0,1


In [10]:
# for example:
df[df['order_name']=='SL678614']

,month,product_type,order_id,order_name,net_sales,ordered_item_quantity
1038718,2021-01,NaN,2978292269149,SL678614,20.0,1


this could be some peculiar transaction (e.g. a return or an unlisted product)

In [11]:
# removing them:
df = df[~df['product_type'].isna()]
df.isna().sum()

month                    0
product_type             0
order_id                 0
order_name               0
net_sales                0
ordered_item_quantity    0
dtype: int64

In [12]:
# example of a basket purchase
df[df['order_name']=='SL672715']

,month,product_type,order_id,order_name,net_sales,ordered_item_quantity
1045978,2021-01,Mask Girls,2959242494045,SL672715,5.25,1
1046375,2021-01,Mask With Nose Shape,2959242494045,SL672715,10.50,2
1060961,2021-01,Mask Lanyard,2959242494045,SL672715,5.25,1
1064417,2021-01,Kids Mask,2959242494045,SL672715,10.50,2


In [13]:
df.groupby(['order_name','product_type'])['ordered_item_quantity'].sum()
#.unstack().reset_index().fillna(0).set_index('order_name'))

order_name  product_type        
SL271975    Head N Nek              10
            Sleeve                   2
SL271976    Tights                   4
SL271977    Sleeve                   2
SL271978    3/4 Tights               1
                                    ..
SL764177    Chin Strap Cover         1
            Spats                    1
SL764178    Sleeve                   2
SL764179    Football Leg Sleeves     1
            Spats                    2
Name: ordered_item_quantity, Length: 738371, dtype: int64

In [14]:
# Unstacking and One-Hot Encoding by 'order_name'
df.groupby(['order_name','product_type'])['ordered_item_quantity'].sum().unstack().fillna(0)

product_type,2-Layer Neck Gaiter,3/4 Tights,Arm Shiver,Back Plate Decal,Basics Shirt,Bat Grip,Batting Gloves,Bicep Band,Bracelet,Bucket Hats,...,Visor Skin,Visor with Skin,Wide Headband,Windbreaker,Winter Headband,Wristband,Yoga Pants,customization service,treDCAL,zakeke-design
order_name,,,,,,,,,,,,,,,,,,,,,
SL271975,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
SL271976,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
SL271977,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
SL271978,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
SL271979,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
SL764175,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
SL764176,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
SL764177,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [15]:
basket = df.groupby(['order_name','product_type'])['ordered_item_quantity']\
.sum()\
.unstack()\
.reset_index()\
.fillna(0)\
.set_index('order_name')

In [16]:
basket.head(3)

product_type,2-Layer Neck Gaiter,3/4 Tights,Arm Shiver,Back Plate Decal,Basics Shirt,Bat Grip,Batting Gloves,Bicep Band,Bracelet,Bucket Hats,...,Visor Skin,Visor with Skin,Wide Headband,Windbreaker,Winter Headband,Wristband,Yoga Pants,customization service,treDCAL,zakeke-design
order_name,,,,,,,,,,,,,,,,,,,,,
SL271975,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
SL271976,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
SL271977,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [17]:
basket[basket.index=='SL672715']

product_type,2-Layer Neck Gaiter,3/4 Tights,Arm Shiver,Back Plate Decal,Basics Shirt,Bat Grip,Batting Gloves,Bicep Band,Bracelet,Bucket Hats,...,Visor Skin,Visor with Skin,Wide Headband,Windbreaker,Winter Headband,Wristband,Yoga Pants,customization service,treDCAL,zakeke-design
order_name,,,,,,,,,,,,,,,,,,,,,
SL672715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [18]:
basket[basket.index=='SL672715'].sum(axis=1)

order_name
SL672715    6.0
dtype: float64

In [19]:
# convert the units to 1 hot-encoding values

def encode_units(x):
    if x <= 0:
        return 0
    if x >= 1:
        return 1
    

In [20]:
basket_sets = basket.applymap(encode_units)
basket_sets.head(3)

product_type,2-Layer Neck Gaiter,3/4 Tights,Arm Shiver,Back Plate Decal,Basics Shirt,Bat Grip,Batting Gloves,Bicep Band,Bracelet,Bucket Hats,...,Visor Skin,Visor with Skin,Wide Headband,Windbreaker,Winter Headband,Wristband,Yoga Pants,customization service,treDCAL,zakeke-design
order_name,,,,,,,,,,,,,,,,,,,,,
SL271975,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
SL271976,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
SL271977,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [21]:
basket[basket.index=='SL672715'].sum(axis=1)

order_name
SL672715    6.0
dtype: float64

In [22]:
ex_row = (basket.loc[basket.index=='SL672715',:]>0)
ex_row = ex_row.values[0]
ex_row

array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False,  True, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False,  True,
       False,  True, False, False, False, False, False, False,  True,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False,

In [23]:
ex_cols = np.array(basket.columns)
ex_cols

array(['2-Layer Neck Gaiter', '3/4 Tights', 'Arm Shiver',
       'Back Plate Decal', 'Basics Shirt', 'Bat Grip', 'Batting Gloves',
       'Bicep Band', 'Bracelet', 'Bucket Hats', 'Calf Sleeves', 'Charm',
       'Chest Pack', 'Chin Strap Cover', 'Comfy Mask',
       'Compression Hoodie', 'Compression Jersey', 'Cooling Towel',
       'Cotton Bicep Bands', 'Cotton Headband', 'Cotton Wristband',
       'Credit Box', 'Custom', 'DD56', 'Demario Davis',
       'Dry Fit Neck Gaiter', 'Durag', 'Ear Saver', 'Fanny Pack',
       'Football Arm Sleeve', 'Football Back Plate', 'Football Headband',
       'Football Leg Sleeves', 'Football Towel', 'Goggle Pouch',
       'Goggle Strap', 'Goggles', 'Hat', 'Head N Nek', 'Headband',
       'Hoodie', 'Horned Headband', 'Jersey', 'Joggers',
       'Kids Arm Shiver', 'Kids Compression Jersey',
       'Kids Football Leg Sleeves', 'Kids Headband', 'Kids Knee Pads',
       'Kids Leg Sleeve', 'Kids Mask', 'Kids Neck Gaiter',
       'Kids Padded Arm Sleeve', 'Kid

In [24]:
ex_row*ex_cols

array(['', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '',
       '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '',
       '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '',
       'Kids Mask', '', '', '', '', '', '', '', '', '', '', '', '', '',
       '', '', '', '', '', '', '', 'Mask Girls', '', 'Mask Lanyard', '',
       '', '', '', '', '', 'Mask With Nose Shape', '', '', '', '', '', '',
       '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '',
       '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '',
       '', '', '', '', '', '', '', '', '', '', ''], dtype=object)

In [25]:
basket[basket.index=='SL672715'][['Mask Girls','Mask Lanyard','Kids Mask','Mask With Nose Shape']]

product_type,Mask Girls,Mask Lanyard,Kids Mask,Mask With Nose Shape
order_name,,,,
SL672715,1.0,1.0,2.0,2.0


### Applying 'apriori' from Mlxtend

In [48]:
frequent_itemsets = apriori(basket_sets, min_support=0.01,use_colnames=True)

In [27]:
frequent_itemsets.head(5)

,support,itemsets
0,0.077361,(Mask With Nose Shape)
1,0.122038,(Neck Gaiter)
2,0.209109,(Sleeve)
3,0.114823,(Spats)
4,0.184880,(Wide Headband)


In [49]:
frequent_itemsets.sort_values('support',ascending=False)

,support,itemsets
21,0.209109,(Sleeve)
28,0.184880,(Wide Headband)
18,0.122038,(Neck Gaiter)
23,0.114823,(Spats)
15,0.077361,(Mask With Nose Shape)
27,0.061405,(Visor)
7,0.050740,(Football Leg Sleeves)
25,0.048477,(Tights)
14,0.046219,(Mask With Filter Pocket)
22,0.039968,(Socks)


In [29]:
# example of basket of goods with the highest support level:
(frequent_itemsets.sort_values(by='support',ascending=False))['itemsets'][:5]

2                  (Sleeve)
4           (Wide Headband)
1             (Neck Gaiter)
3                   (Spats)
0    (Mask With Nose Shape)
Name: itemsets, dtype: object

#### Creating Association Rules

In [57]:
rules = association_rules(frequent_itemsets,metric='lift',min_threshold = 0.001)

In [58]:
rules.sort_values(by='support',ascending=False)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
8,(Wide Headband),(Sleeve),0.184880,0.209109,0.036089,0.195203,0.933497,-0.002571,0.982721
9,(Sleeve),(Wide Headband),0.209109,0.184880,0.036089,0.172585,0.933497,-0.002571,0.985141
6,(Spats),(Sleeve),0.114823,0.209109,0.026985,0.235012,1.123876,0.002974,1.033861
7,(Sleeve),(Spats),0.209109,0.114823,0.026985,0.129047,1.123876,0.002974,1.016331
14,(Spats),(Wide Headband),0.114823,0.184880,0.018291,0.159296,0.861622,-0.002938,0.969569
15,(Wide Headband),(Spats),0.184880,0.114823,0.018291,0.098934,0.861622,-0.002938,0.982367
2,(Mask With Filter Pocket),(Mask With Nose Shape),0.046219,0.077361,0.016502,0.357035,4.615161,0.012926,1.434975
3,(Mask With Nose Shape),(Mask With Filter Pocket),0.077361,0.046219,0.016502,0.213307,4.615161,0.012926,1.212394
0,(Mask With Nose Shape),(Kids Mask),0.077361,0.035569,0.011733,0.151663,4.263953,0.008981,1.136850
1,(Kids Mask),(Mask With Nose Shape),0.035569,0.077361,0.011733,0.329865,4.263953,0.008981,1.376795


In [32]:
# selecting only the top associations
rules[ (rules['confidence'] >= .8) & (rules['lift'] >= 6) ]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction


In [61]:
# Proof via Quantities sold:
display( basket[''].sum() ) # it is summing the number of invoices
display( basket[''].sum() )

#### Exporting

In [62]:
writer = pd.ExcelWriter('Association_byOrder.xlsx', engine='xlsxwriter')
rules.to_excel(writer, sheet_name= 'byOrder', index=False)

In [63]:
writer.save()